In [14]:
import os
import torch
import torchvision

import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn

from torchvision import datasets
from torchvision import models
from torch.utils.data.dataloader import DataLoader
from pathlib import Path
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()


#### Model

In [16]:
class Net(nn.Module):
    def __init__ (self):
        super(Net,self).__init__()
        self. conv1 = nn. Sequential (
            nn.Conv2d (3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d (32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d (2, 2) # output: 64 × 16 × 16
        )
        self. conv2 = nn. Sequential (
            nn.Conv2d (64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d (128, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d (2, 2) # output: 64 × 16 × 16
        )
        self. conv3 = nn. Sequential (
            nn.Conv2d (128, 256, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d (256, 256, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d (2, 2) # output: 64 × 16 × 16
        )
    #仝𨑳层
        self. dense = nn. Sequential (
            nn.Linear (256*4*4, 1024),
            nn.ReLU(),
            nn.Linear (1024, 512),
            nn.ReLU(),
            nn.Linear (512, 10)
        )

    def forward(self,x):
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = out.view(-1,4*4*256)
        out = self.dense(out)
        return out

In [17]:
torch.cuda.set_device(0)
model = Net().cuda()

#### 模型结构可视化

In [19]:
input_to_model = torch.rand(1, 3, 32, 32).cuda()
writer.add_graph(model, input_to_model)

#### 图像样本可视化

In [20]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomAffine(0,shear=10,scale=(0.8,1.2)),
    transforms.ColorJitter(brightness=0.2,contrast=0.2,hue=0.2)
    #随机改变一个图像的亮度、对比度、饱和度和色调
])
test_transform = transforms.Compose([
    transforms.ToTensor()
])

In [21]:
train_data = torchvision.datasets.CIFAR10(
    root = './data/CIFAR10',
    train=True,
    download=True,
    transform=train_transform
)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)
test_data = torchvision.datasets.CIFAR10(
    root = './data/CIFAR10',
    train=False,
    download=True,
    transform=test_transform
)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


In [22]:
images,labels=next(iter(test_loader))

##### 单张图片可视化

In [23]:
writer.add_image('single_sample',images[0])

##### 多张样本可视化

In [24]:
img_grid=torchvision.utils.make_grid(images)
writer.add_image('image_grid',img_grid)

#### 训练过程可视化

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.9)
EPOCHS=20
for epoch in range(EPOCHS):
    train_loss=test_loss=0
    train_acc=test_acc=0
    num_correct=0

    model.train()
    print("Epoch {}/{}".format(epoch+1, EPOCHS))
    for data in train_loader:
        train_img, train_label = data# 数据送入定义好的模型
        train_img=train_img.cuda()
        train_label=train_label.cuda()
        train_img = torch.autograd.Variable(train_img)
        train_label = torch.autograd.Variable(train_label)
        train_out = model(train_img).cuda()#输出放回CPU计算
        optimizer.zero_grad()
        training_loss = criterion(train_out, train_label)
        training_loss.backward()# 反向传播
        optimizer.step()
        train_loss += training_loss.data.float()
        _,train_pred = torch.max(train_out.data,1)#第1维度上的最大值，pred为标签序号
        num_correct+=torch.sum(train_pred==train_label.data)
    train_loss=train_loss/len(train_loader)
    train_acc=100*num_correct/len(train_data)
    print(' Train Loss: {:.3f}, Train Acc: {:.2f}%'.format(train_loss, train_acc))

    model.eval()
    num_correct=0 #初始化为0避免被训练过程影响
    for i, (datas, labels) in enumerate(test_loader):
        test_img, test_label = datas,labels# 数据送入定义好的模型
        test_img = torch.autograd.Variable(test_img).cuda()
        test_label = torch.autograd.Variable(test_label).cuda()
        test_out = model(test_img).cuda()
        testing_loss = criterion(test_out, test_label)
        test_loss += testing_loss.data.float()
        _,test_pred = test_out.max(1)
        num_correct+=torch.sum(test_pred==test_label.data)
    test_loss=test_loss/len(test_loader)
    test_acc=100*num_correct/len(test_data)
    print(' Test Loss: {:.3f}, Test Acc: {:.2f}%'.format(test_loss, test_acc))
    
    writer.add_scalars('loss', 
                       {'train_loss':train_loss, 'test_loss': test_loss},
                       epoch)
    writer.add_scalars('accuracy', 
                        {'train_accuracy': train_acc, 'test_accuracy': test_acc}, 
                        epoch)

Epoch 1/20
 Train Loss: 1.801, Train Acc: 32.65%
 Test Loss: 1.406, Test Acc: 47.43%
Epoch 2/20
 Train Loss: 1.370, Train Acc: 50.30%
 Test Loss: 1.244, Test Acc: 54.85%
Epoch 3/20
 Train Loss: 1.164, Train Acc: 58.26%
 Test Loss: 1.041, Test Acc: 62.61%
Epoch 4/20
 Train Loss: 1.026, Train Acc: 63.56%
 Test Loss: 0.879, Test Acc: 69.54%
Epoch 5/20
 Train Loss: 0.933, Train Acc: 67.40%
 Test Loss: 0.803, Test Acc: 71.87%
Epoch 6/20
 Train Loss: 0.862, Train Acc: 69.64%
 Test Loss: 0.759, Test Acc: 74.04%
Epoch 7/20
 Train Loss: 0.810, Train Acc: 71.51%
 Test Loss: 0.690, Test Acc: 75.87%
Epoch 8/20
 Train Loss: 0.779, Train Acc: 72.74%
 Test Loss: 0.689, Test Acc: 76.64%
Epoch 9/20
 Train Loss: 0.738, Train Acc: 74.05%
 Test Loss: 0.664, Test Acc: 77.18%
Epoch 10/20
 Train Loss: 0.716, Train Acc: 74.99%
 Test Loss: 0.646, Test Acc: 78.05%
Epoch 11/20
 Train Loss: 0.688, Train Acc: 75.82%
 Test Loss: 0.630, Test Acc: 78.52%
Epoch 12/20
 Train Loss: 0.667, Train Acc: 76.60%
 Test Loss: 0